## Labwork 5 | Convex hull of a set.

### Task 1

In [4]:
import numpy as np
from scipy.spatial import ConvexHull

# Generate set E with n ≥ 10 random points
np.random.seed(0)
E = np.random.randint(0, 100, (15, 2))

# Jarvis March (Gift Wrapping)
def jarvis_march(points):
    n = len(points)
    hull = []

    leftmost = min(points, key=lambda p: (p[0], p[1]))
    point_on_hull = leftmost
    while True:
        hull.append(point_on_hull)
        endpoint = points[0]
        for j in range(1, n):
            if (np.array_equal(endpoint, point_on_hull) or 
                np.cross(points[j] - point_on_hull, endpoint - point_on_hull) > 0):
                endpoint = points[j]
        point_on_hull = endpoint
        if np.array_equal(endpoint, hull[0]):
            break
    return np.array(hull)

# Graham Scan using scipy ConvexHull (faster and robust)
def graham_scan(points):
    hull = ConvexHull(points)
    return points[hull.vertices]

# Compute perimeter and area of polygon
def polygon_perimeter_area(poly):
    d = np.linalg.norm(np.roll(poly, -1, axis=0) - poly, axis=1)
    perimeter = d.sum()
    area = 0.5 * np.abs(np.dot(poly[:,0], np.roll(poly[:,1], -1)) - 
                        np.dot(poly[:,1], np.roll(poly[:,0], -1)))
    return perimeter, area

# Apply both algorithms
hull_jarvis = jarvis_march(E)
hull_graham = graham_scan(E)

# Compute perimeter and area
perimeter_jarvis, area_jarvis = polygon_perimeter_area(hull_jarvis)
perimeter_graham, area_graham = polygon_perimeter_area(hull_graham)

E, hull_jarvis, hull_graham, (perimeter_jarvis, area_jarvis), (perimeter_graham, area_graham)


/var/folders/8z/5xw0ppns7xd8g5prmfwnctw80000gn/T/ipykernel_75352/2185683131.py:20: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  np.cross(points[j] - point_on_hull, endpoint - point_on_hull) > 0):


(array([[44, 47],
        [64, 67],
        [67,  9],
        [83, 21],
        [36, 87],
        [70, 88],
        [88, 12],
        [58, 65],
        [39, 87],
        [46, 88],
        [81, 37],
        [25, 77],
        [72,  9],
        [20, 80],
        [69, 79]]),
 array([[20, 80],
        [67,  9],
        [72,  9],
        [88, 12],
        [70, 88],
        [46, 88],
        [36, 87]]),
 array([[88, 12],
        [70, 88],
        [46, 88],
        [36, 87],
        [20, 80],
        [67,  9],
        [72,  9]]),
 (np.float64(236.04237400249212), np.float64(2918.5)),
 (np.float64(236.04237400249212), np.float64(2918.5)))

### Task 2

In [6]:
# Generate two random point sets E1 and E2 with at least 5 points each
np.random.seed(1)
E1 = np.random.randint(0, 100, (7, 2))
E2 = np.random.randint(0, 100, (8, 2))

# Compute convex hulls
hull_E1 = ConvexHull(E1)
hull_E2 = ConvexHull(E2)
poly_E1 = E1[hull_E1.vertices]
poly_E2 = E2[hull_E2.vertices]

# Function to compute polygon intersection using shapely
from shapely.geometry import Polygon, Point

poly1 = Polygon(poly_E1)
poly2 = Polygon(poly_E2)
intersection = poly1.intersection(poly2)

# Find interior points of E1 and E2 that lie strictly inside the intersection
internal_E1 = [tuple(p) for p in E1 if intersection.contains(Point(p))]
internal_E2 = [tuple(p) for p in E2 if intersection.contains(Point(p))]

# Outputs
E1, E2, poly_E1, poly_E2, intersection.wkt, internal_E1, internal_E2


ModuleNotFoundError: No module named 'shapely'